In [1]:
# Import the os package, used to set env variables and check paths
import os
# import the dotenv package for client keys in .env file
from dotenv import load_dotenv
# for random UUID keys
# https://docs.python.org/3/library/uuid.html
import uuid
# Import square 
from square.client import Client
# for pretty print
import pprint

In [2]:
# Get the current working directory
cwd = os.getcwd()

# Construct the .env file path
env_path = os.path.join(cwd, '.env')

# Load the .env file
load_dotenv(dotenv_path=env_path)

True

In [3]:
%%bash
#echo "SQUARE_ACCESS_TOKEN: ${SQUARE_ACCESS_TOKEN}"
#echo "SQUARE_APP_ID: ${SQUARE_APP_ID}"
#git clone https://github.com/square/square-python-sdk.git
#cd square-python-sdk
#pip install .


@hidden_cell

In [4]:
# init the square API
# using generic os environment this fails.
# However, when using the dotenv package it works
sq_client = Client(
    access_token=os.environ['SQUARE_ACCESS_TOKEN'],
    environment='production'  
)


# Create a new item and associate with a new image

## step 1 create a catalog item and add to library

In [35]:
# create a key
uniq_uuid = uuid.uuid4()
#print("uniq_uuid: ", uniq_uuid)

result = sq_client.catalog.upsert_catalog_object(
  body = {
    "idempotency_key": str(uniq_uuid),
    "object": {
      "type": "ITEM",
      "id": "#nickfurydoll",
      "item_data": {
        "name": "Nick Fury Doll",
        "description": "something about Nick Fury from vertex ai",
        "abbreviation": "DFDD",
        "variations": [
          {
            "type": "ITEM_VARIATION",
            "id": "#somenickfuryvariation",
            "item_variation_data": {
              "item_id": "#nickfurydoll",
              "name": "Small doll",
              "pricing_type": "FIXED_PRICING",
              "price_money": {
                "amount": 300,
                "currency": "USD"
              }
            }
          },
        ]
      }
    }
  }
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)

{'catalog_object': {'type': 'ITEM', 'id': 'SVOC3E6ATAISO4SJPPYOSC4N', 'updated_at': '2023-09-24T14:34:36.231Z', 'created_at': '2023-09-24T14:34:36.231Z', 'version': 1695566076231, 'is_deleted': False, 'present_at_all_locations': True, 'item_data': {'name': 'Nick Fury Doll', 'description': 'something about Nick Fury from vertex ai', 'abbreviation': 'DFDD', 'is_taxable': True, 'variations': [{'type': 'ITEM_VARIATION', 'id': 'DKS3MZBOE34DQE7FP6K57D5R', 'updated_at': '2023-09-24T14:34:36.231Z', 'created_at': '2023-09-24T14:34:36.231Z', 'version': 1695566076231, 'is_deleted': False, 'present_at_all_locations': True, 'item_variation_data': {'item_id': 'SVOC3E6ATAISO4SJPPYOSC4N', 'name': 'Small doll', 'ordinal': 0, 'pricing_type': 'FIXED_PRICING', 'price_money': {'amount': 300, 'currency': 'USD'}, 'sellable': True, 'stockable': True}}], 'product_type': 'REGULAR', 'description_html': '<p>something about Nick Fury from vertex ai</p>', 'description_plaintext': 'something about Nick Fury from ver

In [42]:
# print and save the object id
pp = pprint.PrettyPrinter(indent=4)
#pp.pprint(result.body)
# Save off the ids
result_nick_fury = result
nick_fury_cat_obj_id = result_nick_fury.body['catalog_object']['id']
print(nick_fury_cat_obj_id)
# type is 'ITEM'
#print(nick_fury_cat_obj_id['type'])

SVOC3E6ATAISO4SJPPYOSC4N


## Step 2 upload an image an associate with the catalog item


In [44]:
# create a key
uniq_uuid = uuid.uuid4()
#print("uniq_uuid: ", uniq_uuid)

file_to_upload_path = "./sample_imgs/nickfury.jpeg" # Modify this to point to your desired file.
f_stream = open(file_to_upload_path, "rb")



result = sq_client.catalog.create_catalog_image(
  request = {
    "idempotency_key": str(uniq_uuid),
     "object_id": nick_fury_cat_obj_id,
    "image": {
      "type": "IMAGE",
      "id": "#image_id",
      "image_data": {
        "name": "NickFury Doll Image",
        "caption": "image caption nick fury"
      }
    }
  },
  image_file = f_stream
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)

{'image': {'type': 'IMAGE', 'id': 'YZBW4KPTBQT2MRH6AJ3OTQWF', 'updated_at': '2023-09-24T14:41:17.472Z', 'created_at': '2023-09-24T14:41:17.472Z', 'version': 1695566477472, 'is_deleted': False, 'present_at_all_locations': True, 'image_data': {'name': 'NickFury Doll Image', 'url': 'https://items-images-production.s3.us-west-2.amazonaws.com/files/2e7be50da564218c0327c39300dfd2198445bd2d/original.jpeg', 'caption': 'image caption nick fury'}}}


# verify the catalog has the item we added

In [46]:
result = sq_client.catalog.list_catalog()

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)



{'objects': [{'type': 'CUSTOM_ATTRIBUTE_DEFINITION', 'id': 'TN6I6F7QXTEKDRT4M5F6JA7J', 'updated_at': '2023-09-17T16:01:13.582Z', 'created_at': '2023-09-17T16:01:13.582Z', 'version': 1694966473582, 'is_deleted': False, 'present_at_all_locations': True, 'custom_attribute_definition_data': {'type': 'BOOLEAN', 'name': 'Is Alcoholic', 'description': 'Enabling this toggle on an item indicates that it contains alcohol.', 'source_application': {'application_id': 'sq0idp-w46nJ_NCNDMSOywaCY0mwA', 'name': 'Square Online Store'}, 'allowed_object_types': ['ITEM'], 'seller_visibility': 'SELLER_VISIBILITY_HIDDEN', 'app_visibility': 'APP_VISIBILITY_HIDDEN', 'key': 'is_alcoholic'}}, {'type': 'CUSTOM_ATTRIBUTE_DEFINITION', 'id': 'KKJDEQ5ET5QY232B3XO4GRR2', 'updated_at': '2023-09-17T16:01:13.869Z', 'created_at': '2023-09-17T16:01:13.869Z', 'version': 1694966473869, 'is_deleted': False, 'present_at_all_locations': True, 'custom_attribute_definition_data': {'type': 'STRING', 'name': 'Ecom Storefront Classi